In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"
    }
url = "https://websmiley.at.dk/websmiley/advancedsearchform.aspx"

r = requests.get(url,headers=header)

if r.status_code == 200:
    soup = BeautifulSoup(r.content, "html.parser")
    
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    eventvalidation = soup.select("#__EVENTVALIDATION")[0]['value']
    
    
    formData = {
        '__EVENTVALIDATION': eventvalidation,
        '__VIEWSTATE': viewstate,
        '__EVENTTARGET': '',
        'companyName': 'Virksomhedens navn',
        'cvrNr': 'CVR-nr.',
        'pNr': 'P-nr.',
        'ddlBrancheGrupper':0,
        'ddlRegioner':1083,
        'ddlSmiley':0,
        'btnDownload': 'Download smileydata'
        
        
    }
    

r = requests.post(url, data=formData, headers=header)



In [27]:
text = r.text

df = pd.read_csv(StringIO(text),sep=";",keep_default_na=False)

df.columns.tolist()

roede_smileyer = df[(df[" SMILEY"] == 'rød')&(df[" POSTNR"].astype(int) > 4999)&(df[" POSTNR"].astype(int) < 6000)]

roede_smileyer = roede_smileyer.sort_values(" SMILEY_DATO", ascending=False)

roede_smileyer.to_csv("roede_smileyer_paa_fyn.csv")